In [ ]:
#-----------------------------------------------------------------------------------------------------------------------------
#
# Loan Eligibility Classification Model
#
# The script creates a logistic regression model to calculate whether an applicant is eligible for a loan.
# This based on a number of catagorical and continous data fields
# Some of the fields are calculated new independant variables that may give better linear dependancies (potentially) 
# more inisightful (such as Loan to Income)

# The training data set was analysed (in the Loan_History_ML_Eval module) to understand the data
# The script uses shared functions (defined in fn_Aux_Loan_History) to process the data

# The training and test data are loaded

# The user then defines
# .... the subset of parameters to use, 
# .... the columns where null values are filled (catagorical - most poular, numeical mean)
# .... the cost function to evalaute the performance of the model (of two in library set)

# The model is parameterised based on a set of training data
# The training is done by a series of runs (bootstrapping)
# ... splitting the data into a training and verification set
# ... for a number of variations of regulaisation parameter C (a high value fits the data less tightly)
# ... a score is given based on the fraction of values defined as accepted which are true and false (TP and FN)
# ... as this will determine the cost (weighted by the user) of a false acceptance and the benefit of a true acceptance
# ... the training identification output gives the results for every model run

# The training output is analysed (graphically) to asses the regulaisation parameter C that gives the best model (cost)

# The model is created using by running all the training data with the latest tuning data
# The tuning coefficents are presented to give an insight into how the parameter selection can be adapted

# The model is reran against the test data provided to indicate which new applicants should be recomended for approval
# The results are saved (if option is selected) as a pickle for storage and further analysis

# The interntion is that the script is ran iteratively with the parameter combinations selected by the user
# This could be modified (in future) to allow combinations of parameters to be ran in a batch

In [ ]:
#-----------------------------------------------------------------------------------------------------------------------------
# Rev     By                Description
# 1.1    Richard Brooks     Tidy Up
# 1.0    Richard Brooks     Initial Release

sScript = 'Loan History Modelling'
sVersion = 'v1.1'
sAuthor = 'Richard W Brooks'

print ('Running : ' + sScript + ' : ' + sVersion)

In [ ]:
# Importing python libraries and running file with bespoke functions

import numpy as np
import pandas as pd
import pickle
import re
import datetime as dt
import random as rd
import scipy.stats as st

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, precision_score, recall_score, roc_auc_score
from sklearn.preprocessing import MinMaxScaler

# import warnings filter
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# -------------------------------------------------------------------------------------------------------------------------
# 0. Loading and defining functions
# -------------------------------------------------------------------------------------------------------------------------

In [ ]:
run fn_Aux_Loan_History.ipynb

In [ ]:
# Functions options for the cost function are defined as dictionaries so that the corresponding headings can be extracted
# These input actual and predicted values, (and if required parameters)
# They output a score - conistent with teh header

# Weighted Average Cost
# Gives a weighted average score based on a set of weights
# Inputs : weighting given to ['TN', 'FP', 'FN', 'TP'] 
# Outputs : prediction ['TN', 'FP', 'FN', 'TP'], 'T' (number predicted as true), 'SCORE' (weighted score, based on input)
# ... caluates the confusion metric coverts to fraction, then True and Weighted Score based on array input

ls_Weight_Model_Score_Fields = ['TN', 'FP', 'FN', 'TP', 'T', 'SCORE']

def fn_ls_Weight_Model_Score(y_act, y_pred, *args):
    
        ls_weight = args[0]
        valid_confusion = confusion_matrix(y_act, y_pred, labels=[0,1])
        ls_vc = valid_confusion.flatten().tolist()
        ls_vc = [i/sum(ls_vc) for i in  ls_vc]
        
        t = sum([x * y for x, y in zip(ls_vc, [0,0,1,1])])
        ws = sum([x * y for x, y in zip(ls_vc,ls_weight)])
        
        ls_sc = ls_vc + [t, ws]
        
        return ls_sc
    
dict_Weight_Model_Score = {'ls_Fields':ls_Weight_Model_Score_Fields, 'fn' : fn_ls_Weight_Model_Score}

In [ ]:
# Area Under Curve
# Gives the area under the False Possotive / True Possotive Curve - the closer to 1 the better the model
# Inputs : no additional
# Outputs : prediction ['Precsion : TP/(TP+FP)', 'Recall : TP/(TP+FN)', Area Under Curve)

ls_AUC_Score_Fields = ['PREC', 'RECALL', 'SCORE']

def fn_ls_AUC_Score(y_act, y_pred, *args):

        p = precision_score(y_act, y_pred)
        r = recall_score(y_act, y_pred)
        auc = roc_auc_score(y_act, y_pred)   
        ls_sc = [p, r, auc]
        
        return ls_sc
    
dict_AUC_Model_Score = {'ls_Fields':ls_AUC_Score_Fields, 'fn' : fn_ls_AUC_Score}

In [ ]:
# -------------------------------------------------------------------------------------------------------------------------
# 1. Loading the raw data and pre-processing
# -------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Load the raw data sets provided
df_test_raw = pd.read_csv('test_Y3wMUE5_7gLdaTN.csv')
df_train_raw = pd.read_csv('train_u6lujuX_CVtuZ9i.csv')

df_test = df_test_raw.copy()
df_train = df_train_raw.copy()

In [ ]:
# Pre processing the data

# Translate the Output in the training set to a 0/1 value for training
df_train['Loan_Status'] = (df_train['Loan_Status']=='Y').astype(int)

# Change the format of fields and create calculated parameters for inputs (defined in auxillary functions)
df_PreProcess_Data(df_train)
df_PreProcess_Data(df_test)

In [ ]:
df_train.head(n=5)

In [ ]:
df_test.head(n=5)

In [ ]:
# Calculate the fraction of fields in training and test data that have NaN values
# Show this to inform the user for filling NA values

# Columns with null values
iRow = df_test.shape[0]
df_test_NA = df_test.isna().sum().sort_values()
df_test_NA = df_test_NA[df_test_NA>0]/iRow

# Columns with null values
iRow = df_train.shape[0]
df_train_NA = df_train.isna().sum().sort_values()
df_train_NA = df_train_NA[df_train_NA>0]/iRow

df_train_test_NA = df_train_NA.to_frame().rename(columns = {0:'train_NA'}).join(df_test_NA.to_frame().rename(columns = {0:'test_NA'}))

df_train_test_NA

In [ ]:
# The paramters with defined types (found by analysis / inspection in the model evaluation)

# The output to be predicted
ls_Col_Out = ['Loan_Status']

# The catagorical variables in the data set
ls_Col_Cat = ['Dependents','Education', 'Property_Area','Gender','Married','Self_Employed','Credit_History']

In [ ]:
# Show the user what columns are in the data set - have these at hand to copy/psate for selection
df_train.columns

In [ ]:
# -------------------------------------------------------------------------------------------------------------------------
# 2. User Inputs
# -------------------------------------------------------------------------------------------------------------------------

In [ ]:
# The user defines how the data is to be modelled
# ... these inputs can be adjusted to refine the model based on results

# The user then defines
# .... the subset of parameters to use, 

# The independant cause varaibles to be chosen for consideration in the model
ls_Col_Indie = ['Education', 'Property_Area','Married','Credit_History','c_JointIncome','c_Loan_per_AppIncome']

# The varaibles where an assumption is made to fill NA values  (catagorical - most poular, numeical mean)
# ... if the varaiable is not in this list then the row containing NA value is deleted
ls_Col_Fill_NA = ['Married', 'Gender', 'Loan_Amount_Term', 'Self_Employed', 'Credit_History']

# .... the cost function to evalaute the performance of the model (of two in library set)
# Comment out the ones not used

# OPTION A: Weighted Average : True Possotive gives income, False Possotive gives loss
# Change here to select the cost function and output columns from list
dict_Model_Cost = dict_Weight_Model_Score
arg_Model_Cost = [0, -1, 0, 1] # Cost Weight -  ['TN', 'FP', 'FN', 'TP']

# OPTION B: Area Under Curve : Fraction of distingusihing good / bad loan
#dict_Model_Cost = dict_AUC_Model_Score
#arg_Model_Cost = np.NaN

# .... the setting for parameter tuning

# Vales of Regularisation Coeffient 'C', large value simpler model 
ls_Reg_COpts = [0.1, 0.5, 1, 2.5, 5, 7.5, 10, 25, 50] 

 # Fraction of data used for training and validation
d_Valid_Data_Perc = 0.2

# Number of runs per evaluation (of co-efficent)
i_BootstrapRuns = 750 

# ... Whether want to save the main results (as a Pickle) and the associated filename
bSaveOutputPickle = True
sOutputFileBase = 'Loan_History_LR_Model'

In [ ]:
# -------------------------------------------------------------------------------------------------------------------------
# 3. Model Training (Bootstrap)
# -------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Process the Training Data for Model Evaluation

# Select the columns in the training data to use
# Process the NA values - if they are in the list select ... delete the rows for values with remaining NA values
# Normalise the data - so all is between 0 and 1, catagorical data regers to the fields with 1 if matches, 0 of not

ls_Col = list(set(ls_Col_Out + ls_Col_Indie))
df_train = df_FillNA_Data(df_train, ls_Col, ls_Col_Fill_NA, ls_Col_Cat)

df_train_p = df_Prep_Data(df_train,ls_Col_Cat)

In [ ]:
# Run a set of model evaluations
# Conduct a set of runs where split the training data into two sets (identification and validation)
# ....For the options of regularisation parameters (C):
# ... .... Fit the model based on the the training set, evlaute based on the validation set using the score

ls_Score = []
iSet = 1 # the script is just evaluating one set of parameter selection, this may be more if the script is adapted in future

for iRun in range(i_BootstrapRuns):
    
    # Splitting the data
    df_ident_p, df_valid_p = train_test_split(df_train_p, test_size=d_Valid_Data_Perc)

    #Select the output data
    y_train = df_ident_p[ls_Col_Out].copy().values.ravel()
    y_valid = df_valid_p[ls_Col_Out].copy().values.ravel()

    #The remaining columns are inputs
    ls_Col_x_p = list(set(df_train_p.columns) - set(ls_Col_Out))
    x_train = df_ident_p[ls_Col_x_p]
    x_valid = df_valid_p[ls_Col_x_p]
    
    
    # For each regularisation parameter
    for CVal in ls_Reg_COpts:
        
        # Run the model on the training set
        clf = LogisticRegression(C = CVal, max_iter=1000,solver='lbfgs').fit(x_train, y_train)
        
        # Evaluate the model on the test set and give it a score based on the function
        y_valid_pred = clf.predict(x_valid)
        ls_sc = dict_Model_Cost.get('fn')(y_valid, y_valid_pred, arg_Model_Cost)

        # Keep appending the results for each run, using parameters to define the run and the score
        ls = [iSet, iRun, CVal] + ls_sc
        ls_Score.append(ls)

In [ ]:
# Repeat the scoring based on a dummy classfier as a comparison
# ... In the absence of any data you could just assign at random based on the fraction previously accepted
# ... This therefore just compares two splits of the outputs based on the size of the validation data

ls_Dummy_Score = []

y_train = df_train['Loan_Status'].tolist()
iSample = int(len(y_train )*d_Valid_Data_Perc)
ls_Dummy_Score = []

# Take two data splits and run teh model function
# In the list define the parameter set and C value to 0
for iRun in range(i_BootstrapRuns):
    y_valid = rd.sample(y_train,iSample)
    y_dummy_pred = rd.sample(y_train,iSample)
    ls_sc = dict_Model_Cost.get('fn')(y_valid, y_dummy_pred, arg_Model_Cost)
    
    ls = [0, iRun, 0] + ls_sc
    ls_Dummy_Score.append(ls)    

In [ ]:
# Convert the run score to data list to interogate
# This pulls the varaible list associated with the cost function (that is why store in dictionary)
df_Sc = pd.DataFrame(ls_Score + ls_Dummy_Score, columns=['Pm Set', 'Run', 'C'] + dict_Model_Cost.get('ls_Fields'))

In [ ]:
# -------------------------------------------------------------------------------------------------------------------------
# 4. Model Training Evaluation
# -------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Look at the mean results of the runs to assess average sucess
# Compare on the graph the fraction of true values in the training data (as a check)

df_SC_mean = df_Sc[df_Sc['C']>0].groupby(['C','Pm Set']).mean()
df_SC_mean['T Train'] = (df_train['Loan_Status'].value_counts()/len(df_train))[1]

df_SC_mean.plot(kind='line', y = dict_Model_Cost.get('ls_Fields') + ['T Train'])
plt.tight_layout()

In [ ]:
# Look at the distribution in the results for the individal runs (as an indication of outliers)
# Include the dummy classification C = 0 to compare against
# This could be copied and pasted if wnat to look at more score varaibles

df_Sc_L = pd.melt(df_Sc ,id_vars=['C','Pm Set','Run'], var_name='Field', value_name='Val')
sns.catplot(x = "Val", y = "Field", hue = 'C', kind = "violin", data = df_Sc_L[df_Sc_L['Field'] == 'SCORE'])

In [ ]:
# Select the best model parameter
# The score is the maximum value at 95% lower confidence limit

# This could / should be done by eyeballing the results to where the score falls off
# to better inform the choice of parameter and how it comapred with other runs

ls_Score_CI = []
d_CI = 0.95

for CVal in df_Sc['C'].unique():
    ls = df_Sc['SCORE'][df_Sc['C'] == CVal]
    d_Sc_CI = st.t.interval(alpha=d_CI, df=len(ls)-1, loc=np.mean(ls), scale=st.sem(ls))
    ls_Score_CI.append([CVal, d_Sc_CI[0], d_Sc_CI[1] - d_Sc_CI[0],] )
    
df_Score_CI = pd.DataFrame(ls_Score_CI, columns=['C','SCORE','BAND' ])
df_Score_CI = df_Score_CI[df_Score_CI['C']>0]
df_Score_CI.set_index('C', inplace = True)

CSel = df_Score_CI.idxmax(axis=0)[0]
print ('Regularisation Parameter Selected C : ' + str(CSel))

df_Score_CI

In [ ]:
# -------------------------------------------------------------------------------------------------------------------------
# 5. Run the model on the training and test data
# -------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Run the selected model on the full training data set (so teh model is tuned against the maximum available data)

y_train = df_train_p[ls_Col_Out].copy().values.ravel()

ls_Col_x_p = list(set(df_train_p.columns) - set(ls_Col_Out))
x_train = df_train_p[ls_Col_x_p]

clf_sel = LogisticRegression(C = CSel, max_iter=1000,solver='lbfgs').fit(x_train, y_train)

In [ ]:
# Extract the parameters and display these - to give an indication of how important the parmaeters are
# The ones closest to zero may be removed in the next evlauation

df_Model_Coeff = pd.DataFrame({'Parm' : ls_Col_x_p, 'Coff' : clf_sel.coef_.tolist()[0]}).sort_values('Parm')
df_Model_Coeff.set_index('Parm', inplace = True)

plt.figure()
sns.heatmap(df_Model_Coeff,cmap='coolwarm',annot=True, fmt='.2f', center= 0, linewidths=2, linecolor='black')
b, t = plt.ylim() # discover the values for bottom and top
b += 0.5 # Add 0.5 to the bottom
t -= 0.5 # Subtract 0.5 from the top
plt.ylim(b, t) # update the ylim(bottom, top) values
plt.tight_layout()

In [ ]:
# View the results of the model on the Training data

y_train_pred = clf_sel.predict(df_train_p[ls_Col_x_p])
ls_sc_pred = dict_Model_Cost.get('fn')(df_train_p[ls_Col_Out], y_train_pred, arg_Model_Cost)

df_train_sc = pd.DataFrame({'Field':dict_Model_Cost.get('ls_Fields'), 'Val': ls_sc_pred})
df_train_sc.T

In [ ]:
# -------------------------------------------------------------------------------------------------------------------------
# 6. Generate the prediction for the test data
# -------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Process the test data for running the model

# Handle NaN and normalise values as per the training data
df_test = df_FillNA_Data(df_test, ls_Col_Indie, ls_Col_Fill_NA, ls_Col_Cat)
df_test_p = df_Prep_Data(df_test,ls_Col_Cat)

# Check and handle any columns that are missing - (as there may be some catagory differences)
ls_Add_Col = list(set(df_test_p.columns) - set(df_Model_Coeff.index).intersection(ls_Col_x_p))

if len(ls_Add_Col) == 0:
    print ('INFO : All columns in test set exist in training set')
else:
    print ('WARNING : Adding Additional Columns as NAN' + ', '.join(map(str, ls_Add_Col)))
    for sCol in ls_Add_Col:
        df_test_p[sCol] = np.nan

In [ ]:
# Run the model
y_train_test = clf_sel.predict(df_test_p)   

# Add the output to the test data
df_test_results = df_test.copy()
df_test_results[ls_Col_Out[0]] = y_train_test

In [ ]:
# Provide an indication of the loans that would be approved by applying the model

# ... looks the training data actuals and model predicton
# ... how this compares to the test data - whether this can be explained by higher values of fields and coefficients

print('Training Data : {0:.3f}'.format(y_train.sum()/len(y_train)))
print('Training Data Prediction : {0:.3f}'.format(y_train_pred.sum()/len(y_train_pred)))
print('Test Data Prediction : {0:.3f}'.format(y_train_test.sum()/len(y_train_test)))

In [ ]:
# -------------------------------------------------------------------------------------------------------------------------
# 7. Colate and Save Parameters and Output
# -------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Create a dictionary with the key parameter values
dict_Output = {'clf_sel':clf_sel, 'df_test_results':df_test_results, 'df_train_p':df_train_p,
              'df_Sc' : df_Sc, 'sVersion' : sVersion,
              'ls_Col_Indie' :  ls_Col_Indie, 'ls_Col_Fill_NA' : ls_Col_Fill_NA ,'ls_Col_Cat': ls_Col_Cat}

# Add timestamp to the file
sOutputFileName = sOutputFileBase + '_' + dt.datetime.now().strftime("%m-%d-%y-%H%M%S") + '.pck'

# Save this as pickle so it can be arhived
if bSaveOutputPickle:
    pickle.dump(dict_Output,open(sOutputFileName,"wb"))

In [ ]:
# -------------------------------------------------------------------------------------------------------------------------
# End of Script
# -------------------------------------------------------------------------------------------------------------------------